### Load CSV files, clean and transform the data to a Tidy structure for the model construction

After surveying many data sources we decided to use the U.S Bureau of Economics Analysis (https://www.bea.gov/) were we found economic activity data by state in following reports:
1. Gross Domestic Product (GDP) (https://apps.bea.gov/regional/downloadzip.cfm, filename: SAGDP2N__ALL_AREAS_1997_2021.csv )
2. Personal Income (State and Local) (https://apps.bea.gov/regional/downloadzip.cfm, filename:SAINC50__ALL_AREAS_1948_2021.csv)
3. Personal Consumption Expenditures (PCE) by State (https://apps.bea.gov/regional/downloadzip.cfm, filename: SAPCE1__ALL_AREAS_1997_2021.csv)

We selected the U.S Environmental Protection Agency (https://www.epa.gov/) as source for CO2 emissions by state. We used the following reports:
1. Emissions by Unit and Fuel Type (https://www.epa.gov/system/files/other-files/2022-10/emissions_by_unit_and_fuel_type_c_d_aa_10_2022.zip)
2. 2021 Data Summary Spreadsheets (https://www.epa.gov/system/files/other-files/2022-10/2021_data_summary_spreadsheets.zip)

In [8]:
import pandas as pd
import os
import numpy as np
pd.set_option('display.max.columns', None)
pd.set_option('display.max_rows', 50)

In [2]:
pce = pd.read_csv('SAPCE1__ALL_AREAS_1997_2021.csv')
#remove agg and non-pprofit data
pce = pce[~pce.LineCode.isin([1,2,3,8,13,14,22,23,24])]
pce.TableName = 'PCE'

gdp = pd.read_csv('SAGDP2N__ALL_AREAS_1997_2021.csv')
gdp = gdp[gdp.LineCode==1]
gdp.TableName = 'GDP'

inc = pd.read_csv('SAINC50__ALL_AREAS_1948_2021.csv')
inc = inc[~inc.GeoName.isna()]
inc = inc[inc.LineCode.isin([16,20,50])]
inc.TableName = 'INC'

In [3]:
#the income dataset has information since 1948, we only need from 1997
inc = inc[pce.columns]

In [4]:
#concatenate all three datasets and clean the final data
df = pd.concat([pce,gdp,inc])
df = df[~df['GeoName'].isna()]
df.GeoName = df.GeoName.str.replace('*','')
df.GeoName = df.GeoName.str.strip()
df.Description = df.Description.str.strip()

/var/folders/q_/7x1v0d4x7tn3knm4rt_hm7dr0000gn/T/ipykernel_2762/1108004964.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.GeoName = df.GeoName.str.replace('*','')


In [ ]:
#Transform data to a Tidy structure
cols_to_stack = ['1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021']
df = df.melt(id_vars = ['GeoName','Description'], value_vars = cols_to_stack ,var_name = "Year", value_name = "Measures")

df = df.pivot(index = ['GeoName', 'Year'], columns='Description', values = 'Measures').reset_index()

In [ ]:
#Rename columns
cols = ['GeoName', 'Year', 'Motor vehicles and parts',
       'Furnishings and durable household equipment',
       'Recreational goods and vehicles', 'Other durable goods',
       'Food and beverages purchased for off-premises consumption',
       'Clothing and footwear', 'Gasoline and other energy goods',
       'Other nondurable goods', 'Housing and utilities', 'Health care',
       'Transportation services', 'Recreation services',
       'Food services and accommodations',
       'Financial services and insurance', 'Other services',
       'All industry total', 'Equals: Disposable personal income',
       'Population (persons) 1/']
df = df[cols]

cols = [ s.replace(' ','_') for s in df.columns.tolist()]
df.columns = cols

df.rename(columns={'GeoName':'State','All_industry_total':'GDP', 'Equals:_Disposable_personal_income' : 'Disposable_personal_income',
                   'Population_(persons)_1/':'Population' }, inplace=True)

In [ ]:
#The data corresponding to CO2 emissions are in separate spreadsheets by year
#To open each file and build a dataset
co2 = pd.DataFrame()
directory = '2021_data_summary_spreadsheets/'
for filename in os.listdir(directory):
    if filename.endswith(".xlsx"):
        # print(os.path.join(directory, filename))
        data = pd.read_excel(directory+filename, skiprows=3)
        data['Year'] = filename
        co2 = pd.concat([co2, data])
        print('file', filename, ' done!')
    else:
        continue

co2.Year = co2.Year.str.extract(r'(\d+)')
co2['CO2'] = co2['CO2 emissions (non-biogenic) '] + co2['Biogenic CO2 emissions (metric tons)'].fillna(0)
co2.rename(columns={'Methane (CH4) emissions ':'CH4','Nitrous Oxide (N2O) emissions ':'N2O'} , inplace=True)
co2 = co2.groupby(['Year','State'], as_index=False, sort=True)['CO2','CH4','N2O'].sum().round(2)

In [ ]:
states = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California"
          ,"CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia"
          ,"HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas"
          ,"KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts"
          ,"MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana"
          ,"NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico"
          ,"NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma"
          ,"OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota"
          ,"TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington"
          ,"WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}

co2['State'] = co2['State'].map(states)

In [ ]:
df = df[df.State.isin(states.values())]
co2 = co2[co2.State.isin(states.values())]

final = df.merge(co2, how='left').reset_index(drop=True)

In [ ]:
# To create a linear regression model and estimate C02 emissions by state from 1997 to 2009 using the economic and consumption data as observations
from sklearn.linear_model import LinearRegression

In [ ]:
df = final.copy()
final = pd.DataFrame()
for s in df.State.unique():
    print(s, df[df.State==s].shape)
    df1 = df[df.State==s]

    X = df1[~df1.CO2.isna()].iloc[:,2:-3]
    y = df1[~df1.CO2.isna()].iloc[:,-3:]
    regr = LinearRegression().fit(X,y)
    
    tp = df1[df1.CO2.isna()].iloc[:,2:-3]
    pred = regr.predict(tp)
    df1.loc[df1[df1.CO2.isna()].index,['CO2','CH4','N2O']] = pred
    
    final = pd.concat([final, df1])

In [ ]:
final = final[['State', 'Year', 'Motor_vehicles_and_parts',
       'Furnishings_and_durable_household_equipment',
       'Recreational_goods_and_vehicles', 'Other_durable_goods',
       'Food_and_beverages_purchased_for_off-premises_consumption',
       'Clothing_and_footwear', 'Gasoline_and_other_energy_goods',
       'Other_nondurable_goods', 'Housing_and_utilities', 'Health_care',
       'Transportation_services', 'Recreation_services',
       'Food_services_and_accommodations', 'Financial_services_and_insurance',
       'Other_services', 'GDP', 'Disposable_personal_income', 'Population',
       'CO2']]

In [ ]:
#To find negative values predicted by the linear regression
df = final.copy()
df[df.CO2<=0].State.value_counts()

In [ ]:
#To replace negative values by moving average imputation
for i,v in df.CO2.iloc[::-1].items():
    if v <=0:
        df.loc[i,'CO2'] = sum(df.loc[i+1:i+3,'CO2'])/3

In [ ]:
final.to_csv('tidy_dataset2_v4.csv', index = False, encoding='utf-8')